<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [3]:
#indicator = pd.DataFrame(df, index=df.index)

In [37]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [38]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [39]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [40]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [41]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [42]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [43]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [44]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [45]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [58]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2017,1,1)
  end = dt.datetime.now()
  #end = dt.datetime(2018,1,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [59]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'YNDX'])

In [60]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [61]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [62]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>middle_band):
        if adj_close>upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score


# To Add
# RSI & CCI

In [63]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    score = ema_scr+(macd_signalline_scr*10)+bollinger_scr+sma_scr
    print(stock, ema_scr, (macd_signalline_scr*10), bollinger_scr, sma_scr, score, '\n')
    return score

In [64]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.014984548423225164 -0.048108866990611565 -0.015329128514419588 -0.030375240904840894 -0.07882868798664688 

AAPL 0.012205656713924817 -0.05853914696239748 -0.02727714300480255 -0.04508776431036554 -0.11869839756364076 

SAP 0.03844475568027672 -0.04240729266944854 0.02390305945902549 -0.010692600174978195 0.009247922294875477 

ACN 0.05970502764974883 -0.061105576588600385 0.00399443042739378 -0.030176628023679153 -0.027582746535136926 

VOW.DE 0.12761803847940745 0.007240698372475984 -0.006665327179778191 -0.01915987100325489 0.10903353866885035 

PLUG -0.2094100453962744 0.2898742122874143 0.041717597076897595 0.06012433034575726 0.18230609431379474 

MDO.DE 0.035258077401211896 -0.045979550593230334 0.003129735424763469 -0.025247761394539547 -0.032839499161794516 

PYPL 0.020531331083330384 -0.05750210265896283 -0.02804964182704983 -0.051596507649537826 -0.1166169210522201 

SQ -0.005637159160656768 -0.1859125891029012 -0.10322085621722765 -0.14624844375636703 -0.441019048237

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-05-19,27.660000,25.750099,26.309999,27.209999,46126933.0,27.209999,PLUG,26.599868,33.645594,25.666800,42.436000,25.831230,20.113127,30.957873,-1.705899,-2.494647,54.562366,73.898583,False,0.182306
2021-05-19,268.399994,261.200012,265.799988,266.200012,72222.0,266.200012,VOW.DE,268.738805,238.324323,271.400001,229.537001,267.986228,250.870550,283.469453,-3.231980,-3.424728,49.638962,37.802281,False,0.109034
2021-05-19,64.169998,63.145000,63.619999,64.040001,1057195.0,64.040001,YNDX,63.683823,64.279265,63.317601,65.735800,63.682031,60.532410,67.173592,0.061694,0.027785,44.136553,-11.679131,False,0.028652
2021-05-19,137.313797,135.759995,136.220001,136.649994,674679.0,136.649994,SAP,136.057887,131.020823,138.126930,128.633751,136.625649,133.175605,143.367020,1.178770,1.758266,44.966535,-49.826179,False,0.009248
2021-05-19,281.070007,276.885010,279.929993,279.679993,1074890.0,279.679993,ACN,285.580694,269.490742,288.382401,267.530584,286.153993,280.801635,296.357367,0.460798,2.169799,33.263094,-139.286100,False,-0.027583
2021-05-19,189.100006,187.350006,189.100006,188.250000,603.0,188.250000,MDO.DE,191.880741,185.345804,193.125999,182.005049,192.006319,188.841022,197.233976,-0.269530,0.596035,30.729393,-117.136232,False,-0.032839
2021-05-19,224.695007,221.809998,222.759995,224.199997,4674894.0,224.199997,V,225.921892,217.431392,228.020667,215.616572,226.330166,220.724687,236.569763,0.478447,1.514690,36.094655,-74.882373,False,-0.033338
2021-05-19,3232.000000,3185.000000,3195.000000,3217.550049,2090572.0,3217.550049,AMZN,3270.299920,3222.019415,3318.345596,3223.395886,3267.640124,3102.953940,3508.185049,-20.598451,-5.119183,36.259351,-43.520719,False,-0.078829
2021-05-19,245.339996,235.110001,235.860001,243.690002,5812294.0,243.690002,PYPL,251.878062,246.810710,256.947601,253.484751,250.722684,232.258462,276.000541,-4.502452,-3.101184,34.573091,-67.989212,False,-0.116617


In [55]:
df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
df_yd

<ipython-input-55-a76c1241de50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)


,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2017-12-29,1184.000000,1167.500000,1182.349976,1169.469971,2688400.0,1169.469971,AMZN,1163.201569,1078.660762,1171.611191,1051.028398,1168.778775,1139.658422,1198.024561,13.016043,16.257446,49.934652,-17.384584,False,1
2017-12-29,42.647499,42.305000,42.630001,42.307499,103999600.0,40.583443,AAPL,41.154661,39.325812,41.260101,39.241777,41.224168,40.116912,42.326502,0.104261,0.253959,41.226182,-100.857288,False,2
2017-12-29,112.690002,111.989998,112.220001,112.360001,287800.0,105.886017,SAP,106.623525,104.471282,106.727379,104.481306,106.601376,104.169403,108.557272,-0.141477,-0.015683,45.250555,-50.537647,False,1
2017-12-29,154.580002,153.070007,154.429993,153.089996,1452100.0,145.209702,ACN,142.827628,133.993965,142.724578,132.477247,143.523666,138.959076,147.838879,1.916780,2.020138,60.968408,46.325067,False,3
2017-12-29,170.699997,168.300003,170.449997,168.699997,46334.0,154.476273,VOW.DE,154.369032,143.427342,155.591578,138.833604,155.004289,152.357751,159.035094,1.373801,1.949577,49.223987,-78.020679,False,1
2017-12-29,2.440000,2.350000,2.430000,2.360000,3770800.0,2.360000,PLUG,2.415328,2.404294,2.385200,2.451700,2.406621,2.304272,2.485728,-0.016992,-0.025740,43.209027,-45.748591,False,2
2017-12-19,143.399994,143.399994,143.399994,143.399994,0.0,130.878632,MDO.DE,130.236223,125.899100,130.170328,124.904061,130.461464,128.606766,132.162386,0.548693,0.636147,80.685988,-inf,False,3
2017-12-29,74.529999,73.580002,73.919998,73.620003,4496600.0,73.620003,PYPL,74.086555,68.632476,74.387200,68.323000,74.178810,71.477858,76.172142,0.111421,0.231775,48.904369,-36.467425,False,1
2017-12-29,35.080002,34.380001,34.790001,34.669998,10462300.0,34.669998,SQ,37.055218,33.643764,37.858000,32.872400,36.817724,34.254249,39.312750,-1.015452,-0.701168,31.003576,-93.563237,False,1


In [54]:
df_complete[df_complete.dead_cat_bounce==True]

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce
Date,,,,,,,,,,,,,,,,,,,


In [22]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()